# APIM ❤️ FinOps

## FinOps Framework lab
![flow](../../media/common/finops-framework.gif)

This playground leverages the [FinOps Framework](https://www.finops.org/framework/) and Azure API Management to control AI costs. It uses the [token limit](https://learn.microsoft.com/en-us/azure/api-management/azure-openai-token-limit-policy) policy for each [product](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-add-products?tabs=azure-portal&pivots=interactive) and integrates [Azure Monitor alerts](https://learn.microsoft.com/en-us/azure/azure-monitor/alerts/alerts-overview) with [Logic Apps](https://learn.microsoft.com/en-us/azure/azure-monitor/alerts/alerts-logic-apps?tabs=send-email) to automatically disable APIM [subscriptions](https://learn.microsoft.com/en-us/azure/api-management/api-management-subscriptions) that exceed cost quotas.

### Result
![result](result.png)

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [25]:
import os, sys, json
sys.path.insert(1, './shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

aiservices_config = [{"name": "foundry1", "location": "swedencentral"}]

models_config = [ { "name": "gpt-5.2-chat", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "GPT 5.2 chat inp Gl", "outputTokensMeterSku": "GPT 5.2 chat opt Gl" },
                 { "name": "gpt-5.2", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "GPT 5.2 inp Gl", "outputTokensMeterSku": "GPT 5.2 opt Gl" },
                 { "name": "gpt-5-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "GPT 5 Mini Inpt Glbl", "outputTokensMeterSku": "GPT 5 Mini outpt Glbl" },
                 { "name": "gpt-5-nano", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "GPT 5 Nano Inpt Glbl", "outputTokensMeterSku": "GPT 5 Nano outpt Glbl" },
                 { "name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "gpt 4.1 mini Inp glbl", "outputTokensMeterSku": "gpt 4.1 mini Outp glbl" }, 
                { "name": "gpt-4.1", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 200, "inputTokensMeterSku": "gpt 4.1 Inp glbl", "outputTokensMeterSku": "gpt 4.1 Outp glbl" },
                { "name": "DeepSeek-V3.2", "publisher": "DeepSeek",  "version": "1", "sku": "GlobalStandard", "capacity": 100, "inputTokensMeterSku": "V3.2 Inp glbl", "outputTokensMeterSku": "V3.2 Outp glbl"} ]

apim_sku = 'Basicv2'
apim_products_config = [{"name": "platinum", "displayName": "Platinum Product", "tpm": 2000, "tokenQuota": 1000000, "tokenQuotaPeriod": "Monthly", "costQuota": 15 },
                    {"name": "gold", "displayName": "Gold Product", "tpm": 1000, "tokenQuota": 1000000, "tokenQuotaPeriod": "Monthly", "costQuota": 10}, 
                    {"name": "silver", "displayName": "Silver Product", "tpm": 500, "tokenQuota": 1000000, "tokenQuotaPeriod": "Monthly", "costQuota": 5}]
apim_users_config = [ ]
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1", "product": "platinum" },
                    {"name": "subscription2", "displayName": "Subscription 2", "product": "gold" },
                    {"name": "subscription3", "displayName": "Subscription 3", "product": "silver" },
                     {"name": "subscription4", "displayName": "Subscription 4", "product": "silver" } ]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

currency_code = 'USD'

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 00:44:23.693694 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [20]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

output = utils.run("az ad signed-in-user show", "Retrieved az ad signed-in-user", "Failed to get az ad signed-in-user")
if output.success and output.json_data:
    current_user_object_id = output.json_data['id']

    

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 00:42:21.330566 :1s]
👉🏽 Current user: isarar@microsoft.com
👉🏽 Tenant ID: ddcbdc96-6162-4d91-bb0d-066343049ce1
👉🏽 Subscription ID: 79e1d757-ecdb-4dc3-b0b4-035bac76053d
⚙️ Running: az ad signed-in-user show 
✅ Retrieved az ad signed-in-user ⌚ 00:42:26.326037 :4s]


<a id='2'></a>
### 2️⃣ Verify Azure Developer CLI (azd) deployment

This lab uses [Azure Developer CLI (azd)](https://learn.microsoft.com/azure/developer/azure-developer-cli/overview) to deploy resources. 

⚠️ Ensure you have already run `azd up` from the repository root before continuing with this notebook.

In [21]:
# Verify azd deployment exists
output = utils.run("azd env get-values", "Retrieved azd environment", "Failed to get azd environment. Please run 'azd up' first.")

if output.success:
    utils.print_ok("Azure Developer CLI environment is configured")
else:
    utils.print_error("Please run 'azd up' from the repository root to deploy resources before continuing")

⚙️ Running: azd env get-values 
✅ Retrieved azd environment ⌚ 00:42:26.763480 :0s]
✅ Azure Developer CLI environment is configured ⌚ 00:42:26.764024 


<a id='3'></a>
### 3️⃣ Get the environment values from azd

Retrieve the required outputs from the Azure Developer CLI environment.

In [27]:
# Get all environment values from azd
output = utils.run("azd env get-values", "Retrieved azd environment values", "Failed to get azd environment values")

if output.success:
    env_vars = {}
    for line in output.text.splitlines():
        if '=' in line:
            key, value = line.split('=', 1)
            env_vars[key] = value.strip('"')
    
    # Extract required values (using actual azd variable names)
    apim_resource_gateway_url = env_vars.get('apim_gateway_url')
    pricing_dcr_endpoint = env_vars.get('pricingDCREndpoint')
    pricing_dcr_immutable_id = env_vars.get('pricingDCRImmutableId')
    pricing_dcr_stream = env_vars.get('pricingDCRStream')
    apim_service_name = env_vars.get('apim_service_name')
    subscription_quota_dcr_endpoint = env_vars.get('subscriptionQuotaDCREndpoint')
    subscription_quota_dcr_immutable_id = env_vars.get('subscriptionQuotaDCRImmutableId')
    subscription_quota_dcr_stream = env_vars.get('subscriptionQuotaDCRStream')
    
    # Parse APIM subscriptions JSON if available
    apim_subscriptions_json = env_vars.get('apim_subscriptions')
    if apim_subscriptions_json:
        try:
            apim_subscriptions = json.loads(apim_subscriptions_json.replace("'", '"'))
            for subscription in apim_subscriptions:
                subscription_name = subscription['name']
                subscription_key = subscription['key']
                utils.print_info(f"Subscription Name: {subscription_name}")
                utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
        except Exception as e:
            utils.print_warning(f"Could not parse APIM subscriptions: {e}")
    
    # Display retrieved values
    utils.print_info(f"APIM Gateway URL: {apim_resource_gateway_url}")
    utils.print_info(f"APIM Service Name: {apim_service_name}")
    utils.print_info(f"Pricing DCR Endpoint: {pricing_dcr_endpoint}")
    utils.print_info(f"Pricing DCR Immutable ID: {pricing_dcr_immutable_id}")
    utils.print_info(f"Pricing DCR Stream: {pricing_dcr_stream}")
    utils.print_info(f"Subscription Quota DCR Endpoint: {subscription_quota_dcr_endpoint}")
    utils.print_info(f"Subscription Quota DCR Immutable ID: {subscription_quota_dcr_immutable_id}")
    utils.print_info(f"Subscription Quota DCR Stream: {subscription_quota_dcr_stream}")


⚙️ Running: azd env get-values 
✅ Retrieved azd environment values ⌚ 00:54:11.730686 :0s]
👉🏽 APIM Gateway URL: https://apim-kx3lo3f7gbwya.azure-api.net
👉🏽 APIM Service Name: apim-kx3lo3f7gbwya
👉🏽 Pricing DCR Endpoint: https://dcr-pricing-kx3lo3f7gbwya-mkca-swedencentral.logs.z1.ingest.monitor.azure.com
👉🏽 Pricing DCR Immutable ID: dcr-e7f2bc1bf7ba4f9db4cc9851967993d2
👉🏽 Pricing DCR Stream: Custom-Json-PRICING_CL
👉🏽 Subscription Quota DCR Endpoint: https://dcr-quota-kx3lo3f7gbwya-uspl-swedencentral.logs.z1.ingest.monitor.azure.com
👉🏽 Subscription Quota DCR Immutable ID: dcr-aa485b86ad2d48df8d56365c827817d9
👉🏽 Subscription Quota DCR Stream: Custom-Json-SUBSCRIPTION_QUOTA_CL


<a id='pricing'></a>
### 🔍 Display retail pricing info based on the [pricing API](https://learn.microsoft.com/en-us/rest/api/cost-management/retail-prices/azure-retail-prices)



In [23]:
import requests
from tabulate import tabulate 

def build_pricing_table(json_data, table_data):
    for item in json_data['Items']:
        meter = item['meterName']
        table_data.append([item['armRegionName'], item['armSkuName'], item['retailPrice']*1000])

table_data = []
table_data.append(['Region', 'SKU', 'Retail Price'])
for aiservice in aiservices_config:
    aiservice_resource_location = aiservice['location']    
    prices = requests.get(f"https://prices.azure.com/api/retail/prices?currencyCode='{currency_code}'&$filter=serviceName eq 'Foundry Models' and unitOfMeasure eq '1K' and armRegionName eq '{aiservice_resource_location}'")
    if prices.status_code == 200:
        prices_json = prices.json()
        build_pricing_table(prices_json, table_data)
    print(tabulate(table_data, headers='firstrow', tablefmt='psql'))


+---------------+---------------------------------------------+----------------+
| Region        | SKU                                         |   Retail Price |
|---------------+---------------------------------------------+----------------|
| swedencentral | o3 mini 0131 Batch Outp Data Zone           |          2.42  |
| swedencentral | gpt 4.1 nano cached Inp glbl                |          0.025 |
| swedencentral | gpt4omini-rt-aud1217 Outp regnl             |         24.2   |
| swedencentral | gpt-4o-aud-0603-txt Inp DZone               |          2.75  |
| swedencentral | gpt-4o-rt-aud-0603 Outp glbl                |         80     |
| swedencentral | Phi-4-reasoning-Output                      |          0.5   |
| swedencentral | o1-pro Inp regnl                            |        181.5   |
| swedencentral | o3 0416 Batch Outp glbl                     |          4     |
| swedencentral | gpt 4.1 Inp regnl                           |          2.42  |
| swedencentral | gpt-4o-rt-

<a id='4'></a>
### 4️⃣ Load the pricing data into Azure Monitor custom table

👉 This script uses retail price information. Please adjust it to apply a discount or to use a flat rate with PTUs.   
👉 We are multiplying by 1000 to get the retail price per 1K tokens.   
👉 Deploy this script as a [job](https://learn.microsoft.com/en-us/azure/container-apps/jobs?tabs=azure-cli) to run automatically on a predefined schedule.

In [26]:
import requests
from azure.identity import DefaultAzureCredential
from azure.monitor.ingestion import LogsIngestionClient
from azure.core.exceptions import HttpResponseError
from datetime import datetime, timezone

credential = DefaultAzureCredential()
client = LogsIngestionClient(endpoint=pricing_dcr_endpoint, credential=credential, logging_enable=False)

for aiservice in aiservices_config:
    aiservice_resource_location = aiservice['location']
    prices = requests.get(f"https://prices.azure.com/api/retail/prices?currencyCode='{currency_code}'&$filter=serviceName eq 'Foundry Models' and unitOfMeasure eq '1K' and armRegionName eq '{aiservice_resource_location}'")    
    new_prices = requests.get(f"https://prices.azure.com/api/retail/prices?currencyCode='{currency_code}'&$filter=serviceName eq 'Foundry Models' and unitOfMeasure eq '1M' and armRegionName eq '{aiservice_resource_location}'")
    
    if prices.status_code == 200 and new_prices.status_code == 200:
        prices_json = prices.json()
        new_prices_json = new_prices.json()
        
        if prices_json and 'Items' in prices_json and new_prices_json and 'Items' in new_prices_json:
            for deployment in models_config:
                model_name = deployment.get("name")
                
                # Try to find price in new_prices (1M) first - use without multiplying by 1000
                input_tokens_price = next((item['retailPrice'] for item in new_prices_json['Items'] if item.get('skuName') == deployment.get("inputTokensMeterSku")), None)
                output_tokens_price = next((item['retailPrice'] for item in new_prices_json['Items'] if item.get('skuName') == deployment.get("outputTokensMeterSku")), None)
                
                price_source = "1M"
                
                # If not found in new_prices, fallback to prices (1K) and multiply by 1000
                if input_tokens_price is None:
                    input_tokens_price = next((item['retailPrice'] * 1000 for item in prices_json['Items'] if item.get('skuName') == deployment.get("inputTokensMeterSku")), None)
                    price_source = "1K"
                
                if output_tokens_price is None:
                    output_tokens_price = next((item['retailPrice'] * 1000 for item in prices_json['Items'] if item.get('skuName') == deployment.get("outputTokensMeterSku")), None)
                    price_source = "1K"
                
                utils.print_info(f"Adding model {model_name} (source: {price_source}) with input / output tokens price {input_tokens_price} / {output_tokens_price}")
                body = [{ "TimeGenerated": str(datetime.now(timezone.utc)),
                        "Model": model_name,
                        "InputTokensPrice": input_tokens_price,
                        "OutputTokensPrice": output_tokens_price }]
                try:
                    client.upload(rule_id=pricing_dcr_immutable_id, stream_name=pricing_dcr_stream, logs=body)
                    utils.print_ok(f"Upload succeeded for model {model_name}")
                except HttpResponseError as e:
                    utils.print_error(f"Upload failed: {e}")

👉🏽 Adding model gpt-5.2-chat (source: 1M) with input / output tokens price 1.75 / 14.0
✅ Upload succeeded for model gpt-5.2-chat ⌚ 00:44:35.418197 
👉🏽 Adding model gpt-5.2 (source: 1M) with input / output tokens price 1.75 / 14.0
✅ Upload succeeded for model gpt-5.2 ⌚ 00:44:35.716265 
👉🏽 Adding model gpt-5-mini (source: 1M) with input / output tokens price 0.25 / 2.0
✅ Upload succeeded for model gpt-5-mini ⌚ 00:44:36.046444 
👉🏽 Adding model gpt-5-nano (source: 1M) with input / output tokens price 0.05 / 0.4
✅ Upload succeeded for model gpt-5-nano ⌚ 00:44:36.337245 
👉🏽 Adding model gpt-4.1-mini (source: 1K) with input / output tokens price 0.4 / 1.6
✅ Upload succeeded for model gpt-4.1-mini ⌚ 00:44:36.671425 
👉🏽 Adding model gpt-4.1 (source: 1K) with input / output tokens price 2.0 / 8.0
✅ Upload succeeded for model gpt-4.1 ⌚ 00:44:36.940211 
👉🏽 Adding model DeepSeek-V3.2 (source: 1K) with input / output tokens price 0.58 / 1.6800000000000002
✅ Upload succeeded for model DeepSeek-V3.2 ⌚

<a id='5'></a>
### 5️⃣ Load the Subscription Quota into Azure Monitor custom table


In [28]:
import requests
from azure.identity import DefaultAzureCredential
from azure.monitor.ingestion import LogsIngestionClient
from azure.core.exceptions import HttpResponseError
from datetime import datetime, timezone

credential = DefaultAzureCredential()
client = LogsIngestionClient(endpoint=subscription_quota_dcr_endpoint, credential=credential, logging_enable=False)

for subscription in apim_subscriptions_config:
    for product in apim_products_config:
        if product.get("name") == subscription.get("product"):
            cost_quota = product.get("costQuota")
            utils.print_info(f"Adding {subscription.get('name')} with cost quota {cost_quota}")
            body = [{ 
                "TimeGenerated": str(datetime.now(timezone.utc)),
                "Subscription": subscription.get("name"),
                "Email": subscription.get("email"),
                "CostQuota": cost_quota
            }]
            try:
                client.upload(rule_id=subscription_quota_dcr_immutable_id, stream_name=subscription_quota_dcr_stream, logs=body)
                utils.print_ok(f"Upload succeeded for {subscription.get("name")}")
            except HttpResponseError as e:
                utils.print_error(f"Upload failed: {e}")            


👉🏽 Adding subscription1 with cost quota 15
✅ Upload succeeded for subscription1 ⌚ 00:54:24.589905 
👉🏽 Adding subscription2 with cost quota 10
✅ Upload succeeded for subscription2 ⌚ 00:54:24.928378 
👉🏽 Adding subscription3 with cost quota 5
✅ Upload succeeded for subscription3 ⌚ 00:54:25.257599 
👉🏽 Adding subscription4 with cost quota 5
✅ Upload succeeded for subscription4 ⌚ 00:54:25.588795 


<a id='sdk'></a>
### 🧪 Execute multiple runs using the Azure OpenAI Python SDK

👉 We will send requests with random subscription and models. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.

Use test.http to send request.

<a id='workbooks'></a>
### 🔍 Open the dashboard and workbooks in the Azure Portal

👉 The Cost Analysis workbook contains information on the total costs and quotas for each subscription.  
👉 The [Azure OpenAI Insights workbook](https://github.com/dolevshor/Azure-OpenAI-Insights) provides comprehensive details about service and model usage. Credits to [Dolev Shor](https://github.com/dolevshor/Azure-OpenAI-Insights).  
👉 The [Alerts workbook](https://github.com/microsoft/AzureMonitorCommunity/tree/master/Azure%20Services) provides information about the alerts triggered by Azure Monitor.  

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

Use `azd down` to remove all resources.